In [8]:
import pandas as pd
import numpy as np
from pandas_summary import DataFrameSummary
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [9]:
# pip install pyarrow

In [10]:
df_train = pd.read_feather('all_preprocessed_train.fth')
df_test = pd.read_feather('all_preprocessed_test.fth')

In [11]:
df_train.shape, df_test.shape

((1017209, 81), (41088, 80))

In [12]:
df_train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,...,AfterStateHoliday_bool,BeforeStateHoliday_bool,AfterPromo,BeforePromo,SchoolHoliday_bw,StateHoliday_bool_bw,Promo_bw,SchoolHoliday_fw,StateHoliday_bool_fw,Promo_fw
0,1,5,2015-07-31,5263,555,1,1,0,1,2015,...,57,0,0,0,5.0,0.0,5.0,1.0,0.0,1.0
1,2,5,2015-07-31,6064,625,1,1,0,1,2015,...,67,0,0,0,5.0,0.0,5.0,1.0,0.0,1.0
2,3,5,2015-07-31,8314,821,1,1,0,1,2015,...,57,0,0,0,5.0,0.0,5.0,1.0,0.0,1.0
3,4,5,2015-07-31,13995,1498,1,1,0,1,2015,...,67,0,0,0,5.0,0.0,5.0,1.0,0.0,1.0
4,5,5,2015-07-31,4822,559,1,1,0,1,2015,...,57,0,0,0,5.0,0.0,5.0,1.0,0.0,1.0


In [13]:
# Verificación de que se hicieron bien las cosas en testa también
set(df_train.columns) - set(df_test.columns)

{'Customers', 'Sales'}

In [14]:
(df_train['CompetitionDistance'].isna()).sum()

0

## Categorical vars

In [15]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 
            'CompetitionMonthsOpen', 'Promo2Weeks', 
            'StoreType', 'Assortment', 'PromoInterval', 
            'CompetitionOpenSinceYear', 'Promo2SinceYear', 
            'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 
            'StateHoliday_bool_fw', 'StateHoliday_bool_bw', 'SchoolHoliday_fw', 
            'SchoolHoliday_bw']

In [16]:
#DataFrameSummary(df_train[cat_vars]).summary()

In [17]:
DataFrameSummary(df_train[cat_vars]).summary().loc[['uniques', 'types', 'missing']].T

,uniques,types,missing
Store,1115,numeric,0
DayOfWeek,7,numeric,0
Year,3,numeric,0
Month,12,numeric,0
Day,31,numeric,0
StateHoliday,4,categorical,0
CompetitionMonthsOpen,25,numeric,0
Promo2Weeks,26,numeric,0
StoreType,4,categorical,0
Assortment,3,categorical,0


In [18]:
# Reemplaza nulls por "" en categoricas
# for v in cat_vars: df_train.loc[df_train[v].isnull(), v] = ""

In [19]:
# for v in cat_vars: df_test.loc[df_test[v].isnull(), v] = ""

# Continuos Vars

In [20]:
df_train.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Week', 'Day',
       'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval',
       'CompetitionOpenSince', 'Promo2Since', 'State', 'file', 'week', 'trend',
       'Date_y', 'Month_y', 'Day_y', 'file_DE', 'week_DE', 'trend_DE',
       'Date_DE', 'State_DE', 'Month_DE', 'Day_DE', 'file_y',
       'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
       'Dew_PointC', 'MeanDew_PointC', 'Min_DewpointC', 'Max_Humidity',
       'Mean_Humidity', 'Min_Humidity', 'Max_Sea_Level_PressurehPa',
       'Mean_Sea_Level_PressurehPa', 'Min_Sea_Level_PressurehPa',
       'Max_VisibilityKm', 'Mean_VisibilityKm', 'Min_VisibilitykM',
       'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h', 'Max_Gust_SpeedKm_h',
       'Precipitationmm', 'Cl

In [21]:
#Agrego una mas
df_train['AmplitudTermica'] = df_train.apply(lambda x: x.Max_TemperatureC - x.Min_TemperatureC , axis=1)
df_test['AmplitudTermica'] = df_test.apply(lambda x: x.Max_TemperatureC - x.Min_TemperatureC , axis=1)

In [22]:
contin_vars = ['CompetitionDistance', 
               'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC', 
               'Precipitationmm', 'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 
               'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 
               'trend_DE', 'AfterStateHoliday_bool', 'BeforeStateHoliday_bool', 
               'Promo', 'SchoolHoliday', 'StateHoliday_bool']
contin_vars += ['Open', 'AmplitudTermica', 'CompetitionDaysOpen']

In [23]:
DataFrameSummary(df_train[contin_vars]).summary().loc[['uniques', 'types', 'missing']].T

,uniques,types,missing
CompetitionDistance,654,numeric,0
Max_TemperatureC,50,numeric,0
Mean_TemperatureC,45,numeric,0
Min_TemperatureC,40,numeric,0
Precipitationmm,40,numeric,0
Max_Humidity,52,numeric,0
Mean_Humidity,71,numeric,0
Min_Humidity,93,numeric,0
Max_Wind_SpeedKm_h,42,numeric,0
Mean_Wind_SpeedKm_h,27,numeric,0


In [24]:
# Reemplaza nulls por 0 en continuas
# for v in contin_vars: df_train.loc[df_train[v].isnull(), v] = 0
# for v in contin_vars: df_test.loc[df_test[v].isnull(), v] = 0

In [25]:
# Variables no consideradas
columnas_todas = df_train.columns
columnas_sin_asignar = set(df_train.columns) - set(contin_vars)  - set(cat_vars)
columnas_sin_asignar

{'AfterPromo',
 'AfterSchoolHoliday',
 'BeforePromo',
 'BeforeSchoolHoliday',
 'CompetitionOpenSince',
 'CompetitionOpenSinceMonth',
 'Customers',
 'Date',
 'Date_DE',
 'Date_y',
 'Day_DE',
 'Day_y',
 'Dew_PointC',
 'Max_Gust_SpeedKm_h',
 'Max_Sea_Level_PressurehPa',
 'Max_VisibilityKm',
 'MeanDew_PointC',
 'Mean_Sea_Level_PressurehPa',
 'Mean_VisibilityKm',
 'Min_DewpointC',
 'Min_Sea_Level_PressurehPa',
 'Min_VisibilitykM',
 'Month_DE',
 'Month_y',
 'Promo2',
 'Promo2Days',
 'Promo2Since',
 'Promo2SinceWeek',
 'Sales',
 'StateName',
 'State_DE',
 'Week_y',
 'WindDirDegrees',
 'Year_y',
 'file',
 'file_DE',
 'file_y',
 'week',
 'week_DE'}

# Normalización en continuas y LabelEncode en Categoricas

In [26]:
# pip install sklearn_pandas
from sklearn_pandas import DataFrameMapper

In [27]:
cat_maps = [(o, LabelEncoder()) for o in cat_vars]
contin_maps = [([o], StandardScaler()) for o in contin_vars]

In [28]:
mapper_cat = DataFrameMapper(cat_maps)
_ = mapper_cat.fit(df_train)

In [29]:
# Assortment es la novena posición en cat_maps
N = 10
print(list(zip(df_train['Assortment'].values[:N], mapper_cat.transform(df_train)[:,9][:N])))
print(list(zip(df_train['Events'].values[:N], mapper_cat.transform(df_train)[:,15][:N])))
print(list(zip(df_train['Year'].values[:N], mapper_cat.transform(df_train)[:,2][:N])))

[('a', 0), ('a', 0), ('a', 0), ('c', 2), ('a', 0), ('a', 0), ('c', 2), ('a', 0), ('c', 2), ('a', 0)]
[('Fog', 0), ('Fog', 0), ('Fog', 0), ('Sunny', 20), ('Sunny', 20), ('Sunny', 20), ('Rain', 10), ('Rain', 10), ('Fog', 0), ('Sunny', 20)]
[(2015, 2), (2015, 2), (2015, 2), (2015, 2), (2015, 2), (2015, 2), (2015, 2), (2015, 2), (2015, 2), (2015, 2)]


In [30]:
mapper_cont = DataFrameMapper(contin_maps)
_ = mapper_cont.fit(df_train)

In [31]:
N = 10
print(df_train['CompetitionDistance'].values[:N])
print(mapper_cont.transform(df_train)[:, 0][:N])

[ 1270.   570. 14130.   620. 29910.   310. 24000.  7520.  2030.  3160.]
[-0.51104292 -0.59341201  1.00219483 -0.5875285   2.85902934 -0.62400624
  2.16359893  0.22439533 -0.42161363 -0.28864639]


In [32]:
# La hacemos con todas ahora
df_train[cat_vars] = mapper_cat.transform(df_train)
df_test[cat_vars] = mapper_cat.transform(df_test)

In [33]:
df_train[contin_vars] = mapper_cont.transform(df_train)
df_test[contin_vars] = mapper_cont.transform(df_test)

In [34]:
df_train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,...,BeforeStateHoliday_bool,AfterPromo,BeforePromo,SchoolHoliday_bw,StateHoliday_bool_bw,Promo_bw,SchoolHoliday_fw,StateHoliday_bool_fw,Promo_fw,AmplitudTermica
0,0,4,2015-07-31,5263,555,0.452399,1.273237,0,2.144211,2,...,1.072424,0,0,5,0,5,1,0,1,1.516796
1,1,4,2015-07-31,6064,625,0.452399,1.273237,0,2.144211,2,...,1.072424,0,0,5,0,5,1,0,1,0.824404
2,2,4,2015-07-31,8314,821,0.452399,1.273237,0,2.144211,2,...,1.072424,0,0,5,0,5,1,0,1,1.516796
3,3,4,2015-07-31,13995,1498,0.452399,1.273237,0,2.144211,2,...,1.072424,0,0,5,0,5,1,0,1,0.362809
4,4,4,2015-07-31,4822,559,0.452399,1.273237,0,2.144211,2,...,1.072424,0,0,5,0,5,1,0,1,0.362809


In [35]:
DataFrameSummary(df_train[cat_vars]).summary().loc[['uniques', 'types']]

,Store,DayOfWeek,Year,Month,Day,StateHoliday,CompetitionMonthsOpen,Promo2Weeks,StoreType,Assortment,...,State,Week,Events,Promo_fw,Promo_bw,StateHoliday_bool_fw,StateHoliday_bool_bw,SchoolHoliday_fw,SchoolHoliday_bw,Open
uniques,1115,7,3,12,31,4,25,26,4,3,...,12,52,22,6,6,3,3,8,8,2
types,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,...,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,bool


# Final processing

In [36]:
df_train.shape

(1017209, 82)

In [37]:
df_sales = df_train[df_train.Sales!=0]

In [38]:
df_sales.shape

(844338, 82)

In [39]:
df_test.reset_index(inplace=True)

In [40]:
df_sales.reset_index(inplace=True)

In [41]:
df_sales.to_feather('train_normalized_data.fth')

In [42]:
df_test.to_feather('test_normalized_data.fth')

In [43]:
(df_test['Open'] == 0).sum()

0

In [44]:
(df_sales['Open'] == 0).sum()

0

In [45]:
(df_train['CompetitionDistance'].isna()).sum()

0

In [46]:
#import beepy
#beepy.beep(sound=1)

from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)

date and time = 27/02/2023 15:46:24
